In [1]:
# Denpendancies
import os
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
# Softmax
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

# Load tickers from Database
def get_tickers(database='Database'):
    db = os.path.join(os.getcwd(), database)
    if os.path.exists(db):
        tickers = [ld.split('.')[0] for ld in os.listdir(db)]
        return sorted(tickers)
    print('Database could not be found.')

def batching(data, period):
    batch = len(data) // period
    max_length = batch * period
    return np.array(data)[-max_length:].reshape(-1, period)

def regroup(data, period):
    base = batching(data['Open'], period)[:, 0]
    high = batching(data['High'], period).max(1)
    low = batching(data['Low'], period).min(1)
    close = batching(data['Close'], period)[:, -1]
    return pd.DataFrame(np.stack([base, high, low, close], axis=1), 
                        columns=['Open', 'High', 'Low', 'Close'])

In [10]:
# Testing leveled Buy / Sell tricker
# Using Apple as first single test
data = DataLoader('AAPL').get_data('2018-01-01', '2020-12-31')

In [11]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,41.188168,102223600
2018-01-03,43.132500,43.637501,42.990002,43.057499,41.180988,118071600
2018-01-04,43.134998,43.367500,43.020000,43.257500,41.372272,89738400
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.843307,94640000
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.687897,82271200
...,...,...,...,...,...,...
2020-12-24,131.320007,133.460007,131.100006,131.970001,131.161407,54930100
2020-12-28,133.990005,137.339996,133.509995,136.690002,135.852509,124486200
2020-12-29,138.050003,138.789993,134.339996,134.869995,134.043655,121047300


In [12]:
# Calculate Momentum from differnt timeframe
# If wanted to buy and sell at a special period
# vs buy and sell at any avaible time
timeframe = [3, 5, 10, 20]
for tf in timeframe:
    ndata = regroup(data, tf)
    set_period = (ndata['Close'].shift(-1) / ndata['Close'] - 1).dropna()
    any_period = (data['Close'].shift(-tf) / data['Close'] - 1).dropna()
    print('{:.4f} | {:.4f}'.format(set_period.mean(), any_period.mean()))

0.0051 | 0.0051
0.0084 | 0.0085
0.0174 | 0.0169
0.0368 | 0.0341


In [16]:
# Creating gain and loss probability table from closing prices
prob = None
timeframe = [3, 5, 10, 20]
for tf in timeframe:
    ndata = regroup(data, tf)
    gains = (ndata['Close'].shift(-1) / ndata['Close'] - 1).dropna()
    dic = {}
    for p in np.arange(0.001, 0.101, step=0.003):
        # Create Masks
        mask = gains > 0
        a = gains[mask]
        b = gains[~mask]
        a_mean = (a > p).mean()
        b_mean = (b < -p).mean()
        dic[p] = {'Above': a_mean, 'Below': b_mean}
    if prob is None:
        prob = pd.DataFrame.from_dict(dic, orient='index')
    else:
        prob += pd.DataFrame.from_dict(dic, orient='index')

# Softmax probability
prob['Above'] = softmax(prob['Above'])
prob['Below'] = softmax(prob['Below'])
prob

,Above,Below
0.001,0.144253,0.183949
0.004,0.120856,0.137941
0.007,0.102352,0.104261
0.010,0.085477,0.078898
0.013,0.071159,0.067704
0.016,0.055893,0.057993
0.019,0.047956,0.042422
0.022,0.042219,0.033370
0.025,0.032440,0.023389
0.028,0.027971,0.021279


In [17]:
# Creating gain and loss probability table from close to high prices
prob = None
timeframe = [3, 5, 10, 20]
for tf in timeframe:
    ndata = regroup(data, tf)
    gains = (ndata['High'].shift(-1) / ndata['Close'] - 1).dropna()
    dic = {}
    for p in np.arange(0.001, 0.101, step=0.003):
        # Create Masks
        mask = gains > 0
        a = gains[mask]
        b = gains[~mask]
        a_mean = (a > p).mean()
        b_mean = (b < -p).mean()
        dic[p] = {'Above': a_mean, 'Below': b_mean}
    if prob is None:
        prob = pd.DataFrame.from_dict(dic, orient='index')
    else:
        prob += pd.DataFrame.from_dict(dic, orient='index')

# Softmax probability
prob['Above'] = softmax(prob['Above'])
prob['Below'] = softmax(prob['Below'])
prob

,Above,Below
0.001,0.125310,0.372554
0.004,0.103966,0.139358
0.007,0.096391,0.085232
0.010,0.085947,0.078418
0.013,0.075058,0.047961
0.016,0.064974,0.011729
0.019,0.056280,0.011729
0.022,0.047334,0.011250
0.025,0.039518,0.009523
0.028,0.034793,0.009523
